In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df_train.shape

(7613, 5)

In [3]:
import sklearn
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train.text,df_train.target, test_size = 0.2)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
x_train[:4]

1470    @peterjukes But there are good grounds to beli...
6382    16yr old PKK suicide bomber who detonated bomb...
4161    Sex Workers Say Credit Card Bans On Online Ads...
2189    Malaysia confirms plane debris washed up on Re...
Name: text, dtype: object

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

x_train_cv = v.fit_transform(x_train.values)
x_train_np = x_train_cv.toarray()

In [6]:
np.where(x_train_np[0] != 0)

(array([ 1666,  1846,  2453,  3099,  3121,  3372,  3941,  4272,  7103,
         7234,  8332, 10595, 12286, 12447, 12549, 15936, 15998, 16203,
        17394]),)

In [7]:
v.get_feature_names_out()[2732]

'blues'

In [8]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train_cv,y_train)

MultinomialNB()

In [9]:
x_test_cv = v.transform(x_test)
y_pred = model.predict(x_test_cv)
from  sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.87      0.82       855
           1       0.80      0.69      0.74       668

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



In [10]:
from sklearn.pipeline import Pipeline
clf = Pipeline([('vectorizer',CountVectorizer()),('nb',MultinomialNB())])
clf.fit(x_train,y_train)
print(classification_report(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.78      0.87      0.82       855
           1       0.80      0.69      0.74       668

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



In [11]:
test_text_cv = v.transform(df_test.text)
test_text_cv

<3263x18488 sparse matrix of type '<class 'numpy.int64'>'
	with 41058 stored elements in Compressed Sparse Row format>

In [12]:
test_pred = model.predict(test_text_cv)
ID = df_test.id
target = test_pred
Sub = pd.DataFrame(ID, columns = ['id'])
Sub['target'] = target
Sub.to_csv('Submission.csv', index = False)